# Questions

1. Characterize the data and comment about its quality
2. Explore and visualize the data e.g. a histogram of trip distance
3. Find interesting trip statistics grouped by hour
4. The taxi drivers want to know what kind of trip yields better tips. Can you build a model for them and explain the model?
5. Pick one of the options below
    - (Option 1) Find an anomaly in the data and explain your findings.
    - (Option 2) Visualize the data to help understand trip patterns

## 1. Characterize the data and comment about its quality
- After downloading the data create some checks
- Review the data dictionary to see of there are weird things about

## 2. Explore and visualize the data e.g. a histogram of trip distance
- Might combine some of the results from the checks in question 1 with this exploration
- I somewhat group finding interesting stats grouped by hour under this umbrella as well
- Make sure to visualize a histogram of tip distance

## 3. Find interesting trip statistics grouped by hour
- For now I will assume this means the start of the trip. However, some trips can laat longer than an hour
 
## 4. The taxi drivers want to know what kind of trip yields better tips. Can you build a model for them and explain the model?
- This is a very vague request, but maybe after interacting with the data some obvious patterns may show up
- There are some leading questions that suggest they may be interested in "Time of day" as well as the length of the trips
- In the model explanation I would be sure to include that it may be beneficial to come to an agreement on what "better tips" mean.
    - Are we talking about just a higher tip?
    - Should we take into consideration cost of drivers time?
    - Longer trips may earn better tips, but a lower percentage of the entire trip cost.
    - Suggestion: use "tip rate" to define better tips. You can also think of this as Percent of Trip Cost.
        - tip/total cost of trip
            - i.e. {'trip_total_cost': '40.00', 'tip': '8.00'}
            - tip_rate = 8/40 = 0.20
    - We can view all these tips on a distribution and either use explicit classification (informed by project management) or ML to generate insights. A really simple approach would be to split the distribution of tip_rates into quartiles (low, med, high, very high). These types of tip ranges can inform a classification model, maybe KNN, of what future trips might yeild. We have the option of harness testing these models to inform users of which trips yeild better tips through simulation OR we can simply use statistics (A/B tests) on the data provided to inform decision makers of the features that are most likely to impact tips.

## 5. Pick one of the options below
### Find an anomaly in the data and explain your findings.
I think this may be the more likely option.
### Visualize the data to help understand trip patterns